In [29]:
import numpy as np
import pennylane as qml
from pennylane import AdamOptimizer
from tqdm import tqdm
from vqa import transforms
from vqa.templates.examples import h2_vqe_circuit
from vqa.templates.circuits import AbstractCircuit
from vqa.utils.utils import get_approximation_ratio

# Introduction

To understand the workflow of the vqa library we start by solving the ground state estimation for H2. 
 
1. Load circuit template $\ket{\psi(\theta)}$.
2. Wrap it into a [QNode](https://docs.pennylane.ai/en/stable/code/api/pennylane.QNode.html), to generate an executable function that returns e.g. the expecation value $\bra{\psi(\theta)} H \ket{\psi(\theta)}$.
3. Select an optimizer and optimize.

## The quantum circuit template class

Each circuit template contains an executable circuit, the number of wires or qubits needed and an `init()` method. The `init()` returns the parameters to optimize. For reproducability purposes we can seed the `init()` method.

In [30]:
circuit = h2_vqe_circuit(num_layers=1)
params = circuit.init(seed=0)

converged SCF energy = -1.12606474197566


## Creating a loss function for optimization

Next we need to describe what observable we want to apply to the circuit. The most common approach is to calculate the expectation value such that $\bra{\psi(\theta)} H \ket{\psi(\theta)}$.

In [31]:
dev = qml.device("default.qubit", wires=circuit.wires)


@qml.qnode(dev)
def circ(params):
    circuit(params)
    return qml.expval(circuit.H)


circ(params)

tensor(-0.14651095, requires_grad=True)

#### Alternative usage
Alternatively, we can also implement our quantum circuit directly and avoid using the `AbstractCircuit` class. Note that you are required to provide the wires, the Hamiltonian and the parameters manually (This is a random circuit).

In [32]:
def _circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[2, 1])
    qml.RZ(params[2], wires=2)
    qml.RX(params[3], wires=3)
    qml.CNOT(wires=[0, 1])

In [33]:
_params = np.array([0.1, 0.2, 0.3, 0.4])


dev = qml.device("default.qubit", wires=range(4))


@qml.qnode(dev)
def circ(params):
    _circuit(params)
    return qml.expval(qml.PauliZ(0))


circ(_params)

tensor(0.99500417, requires_grad=True)

It should be evident now why the `AbstractCircuit` class is beneficial. It hides unnecessary details from the end user and allows fast prototyping for already implemented circuits.

## Optimizing the circuit

Now we are ready to optimize the circuit. We need a loop to iterate between the classical optimizer and the quantum circuit. 

In [34]:
optimizer = AdamOptimizer(0.1)

for i in tqdm(range(100)):
    params = optimizer.step(fun, params)
    exp_val = fun(params)

print(f"Final energy: {np.round(exp_val,3)}")

100%|██████████| 100/100 [00:02<00:00, 49.71it/s]

Final energy: -0.526


In [35]:
# r = get_approximation_ratio(exp_val, circuit.ground_state_energy, 0)
# print(f"Approximation ratio: {np.round(r, 3)}")